In [1]:
import os
import pickle

from datetime import datetime
from typing import Literal

import torch
import wandb
import numpy as np
import tensorflow as tf
import tensorflow_ranking as tfr
from torch import nn
from torch import optim
import torch_geometric.nn as geonn
from torch_geometric.data import Data, Batch
from tqdm import tqdm

from gfos.data.utils import load_layout
from gfos.data.dataset import LayoutDataset, Normalizer
from gfos.metrics import kendall, topk_error
from gfos.loss import MultiElementRankLoss
from gfos.utils.misc import seed_everything
from gfos.data.constants import mask_min_max

SEED = 42
DEBUG = False

seed_everything(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"

LAYOUT_DIR = r"H:\data\gfos\predict-ai-model-runtime\npz_all\npz\layout"
NORMALIZER_PATH = "../../data/normalizer.json"


In [2]:
SOURCE = "xla"
SEARCH = "random"

normalizer = Normalizer.from_json(NORMALIZER_PATH, SOURCE, SEARCH)

layout_data = load_layout(
    LAYOUT_DIR,
    model_type=SOURCE,
    compile_type=SEARCH,
)

test_dataset = LayoutDataset(
    layout_data["test"],
    normalizer=normalizer,
)

Loading data:   0%|          | 0/8 [00:00<?, ?it/s, 05ae41e26dd3c4c06390371a0423233c]g:\projects\gfos\src\gfos\data\dataset.py:280: RuntimeWarning: invalid value encountered in divide
  runtime = (runtime - mu) / std


Using all configs


Loading data:  12%|█▎        | 1/8 [00:01<00:07,  1.10s/it, 3e7156ac468dfb75cf5c9615e1e5887d]

Using all configs


Loading data:  38%|███▊      | 3/8 [00:02<00:03,  1.33it/s, 5335ed13823b0a518ee3c79ba4425f34]

...

Loading data:  75%|███████▌  | 6/8 [00:02<00:00,  3.58it/s, e8a3a1401b5e79f66d7037e424f3b6df]

Using all configs


Loading data:  75%|███████▌  | 6/8 [00:03<00:00,  3.58it/s, fbaa8bb6a1aed9988281085c91065c05]

Using all configs


Loading data: 100%|██████████| 8/8 [00:03<00:00,  2.10it/s, fbaa8bb6a1aed9988281085c91065c05]


In [4]:
x = test_dataset[0]

No sampling


In [5]:
x

{'model_id': '05ae41e26dd3c4c06390371a0423233c',
 'node_feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]]),
 'node_opcode': tensor([ 63,  63,   2,  ...,  45,  31, 100]),
 'edge_index': tensor([[    2,     2,     5,  ..., 43614, 43614, 43614],
         [    0,     1,     3,  ..., 43609, 43610, 43613]]),
 'node_config_feat': tensor([[[0.6667, 0.5000, 0.3333,  ..., 0.0000, 0.0000, 0.0000],
          [0.6667, 0.1667, 0.0000,  ..., 0.0000, 0.8000, 0.6000],
          [0.6667, 0.3333, 0.1667,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.6667, 0.5000, 0.3333,  ..., 0.000